# ANALYSIS ON USERS
In this code, we want to analyse the **geographical distribution** of **breweries** in order to better understand user distribution data.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import re
import math
from utils import *
from scipy import stats
%pip install plotly
import plotly.express as px


Note: you may need to restart the kernel to use updated packages.


We want to make the code reusable with both datasets, so we will create functions that work on generic entity names, so that we can replicate the analysis while avoiding code redundancy.

In [2]:
import pandas as pd

# Load BeerAdvocate user dataset
ba_users = pd.read_csv('../data/BeerAdvocate/users.csv')

# Load RateBeer user dataset
rb_users = pd.read_csv('../data/RateBeer/users.csv')

# Load Matched Beer Data user dataset
matched_users = pd.read_csv('../data/matched_beer_data/users.csv')

# Print the column names of each DataFrame
print("BeerAdvocate Users Columns:", ba_users.columns.tolist())
print("RateBeer Users Columns:", rb_users.columns.tolist())
print("Matched Users Columns:", matched_users.columns.tolist())

BeerAdvocate Users Columns: ['nbr_ratings', 'nbr_reviews', 'user_id', 'user_name', 'joined', 'location']
RateBeer Users Columns: ['nbr_ratings', 'user_id', 'user_name', 'joined', 'location']
Matched Users Columns: ['ba', 'ba.1', 'ba.2', 'ba.3', 'ba.4', 'ba.5', 'ba.6', 'rb', 'rb.1', 'rb.2', 'rb.3', 'rb.4', 'rb.5']


In [3]:
len(ba_users), len(rb_users), len(matched_users)

(153704, 70174, 3021)

Here we see that matched_users is a really small subset of the data and it probably originated from just joining both dfs using the user_name. Since we want a deeper exploratory analysis, we'll merge the dfs ourselves and keep all entries, even if they only appear in one of the review sites.

In [4]:
# Inspect the unique values in the location column
print("Unique locations in BeerAdvocate Users:")
print(ba_users['location'].unique())

print("\nUnique locations in RateBeer Users:")
print(rb_users['location'].unique())

Unique locations in BeerAdvocate Users:
['United States, Washington' 'United States, New York' 'Northern Ireland'
 'United States, Kansas' 'United States, New Jersey'
 'United States, North Carolina' 'United States, Oklahoma'
 'United States, Pennsylvania' 'United States, California' nan
 'United States, Wisconsin' 'United States, Illinois'
 'United States, Virginia' 'United States, Massachusetts'
 'United States, South Carolina' 'United States, Georgia'
 'United States, Louisiana' 'United States, West Virginia'
 'United States, Alabama' 'United States, New Hampshire'
 'United States, Indiana' 'United States, Rhode Island'
 'United States, Missouri' 'United States, Arkansas' 'United States, Ohio'
 'United States, Connecticut' 'United States, Florida'
 'United States, Minnesota' 'United States, Maryland'
 'United States, Alaska' 'United States, Mississippi'
 'United States, Iowa' 'United States, Oregon' 'United States, Kentucky'
 'England' 'Ireland' 'United States, Michigan' 'Serbia' 'S

In [5]:
#Here we clean the location data by applying the preprocess_location function to each location in the BeerAdvocate and RateBeer datasets
def preprocess_location(df, location_column='location'):
    """
    Preprocess the location column in the given DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the location column.
    location_column (str): The name of the location column to preprocess.
    
    Returns:
    pd.DataFrame: The DataFrame with the preprocessed location column.
    """
    # Convert to lowercase and remove leading/trailing whitespace
    df[location_column] = df[location_column].str.lower().str.strip()
    
    # Replace state-specific locations with 'United States'
    df[location_column] = df[location_column].apply(lambda x: 'united states' if isinstance(x, str) and 'united states' in x else x)
    
    # Correct common misspellings and variations
    # These corrections come from manual inspection of the data and checking that same country is instatiated differently in each dataset
    location_corrections = {
        'england': 'united kingdom',
        'northern ireland': 'united kingdom',
        'scotland': 'united kingdom',
        'wales': 'united kingdom',
        'czech republic': 'czechia',
        'slovak republic': 'slovakia',
    }
    
    df[location_column] = df[location_column].replace(location_corrections)
    
    # Handle missing values by removing rows with missing locations
    df = df.dropna(subset=[location_column])
    
    return df
# Print the length and unique values before preprocessing
print("Before preprocessing:")
print(f"BeerAdvocate Users: {len(ba_users)} rows, {ba_users['location'].nunique()} unique locations")
print(f"RateBeer Users: {len(rb_users)} rows, {rb_users['location'].nunique()} unique locations")

# Clean the location data in the BeerAdvocate and RateBeer datasets
ba_users = preprocess_location(ba_users)
rb_users = preprocess_location(rb_users)

# Print the length and unique values after preprocessing
print("\nAfter preprocessing:")
print(f"BeerAdvocate Users: {len(ba_users)} rows, {ba_users['location'].nunique()} unique locations")
print(f"RateBeer Users: {len(rb_users)} rows, {rb_users['location'].nunique()} unique locations")


Before preprocessing:
BeerAdvocate Users: 153704 rows, 194 unique locations
RateBeer Users: 70174 rows, 222 unique locations

After preprocessing:
BeerAdvocate Users: 122425 rows, 142 unique locations
RateBeer Users: 50592 rows, 169 unique locations


Now we'll merge both dataframes. To do this we'll explore using user_name and user_id to see which achieves a higher overlap of users.

In [8]:
# Standardize columns by adding 'nbr_reviews' to rb_users with default value 0
rb_users['nbr_reviews'] = 0

# Add a source column to each dataset
ba_users['source'] = 'BA'
rb_users['source'] = 'RB'

# Ensure columns are in the same order
ba_users = ba_users[['nbr_ratings', 'nbr_reviews', 'user_id', 'user_name', 'joined', 'location', 'source']]
rb_users = rb_users[['nbr_ratings', 'nbr_reviews', 'user_id', 'user_name', 'joined', 'location', 'source']]

# Convert user_name to lowercase
ba_users['user_name'] = ba_users['user_name'].str.lower().str.strip()
rb_users['user_name'] = rb_users['user_name'].str.lower().str.strip()

# Concatenate the DataFrames
combined_users = pd.concat([ba_users, rb_users], ignore_index=True)

# Handle duplicate users by summing their ratings and reviews using user_name and location
combined_users_by_name = combined_users.groupby(['user_name', 'location']).agg({
    'nbr_ratings': 'sum',
    'nbr_reviews': 'sum',
    'user_id': 'first',  # Keep the first user_id
    'joined': 'first',   # Keep the first joined date
    'source': lambda x: 2 if len(set(x)) > 1 else (0 if 'BA' in x.values else 1)  # Determine the source
}).reset_index()

# Handle duplicate users by summing their ratings and reviews using user_id
combined_users_by_id = combined_users.groupby('user_id').agg({
    'nbr_ratings': 'sum',
    'nbr_reviews': 'sum',
    'user_name': 'first',  # Keep the first user_name
    'joined': 'first',     # Keep the first joined date
    'location': 'first'    # Keep the first location
}).reset_index()

# Print the combined DataFrame
print("Combined Users by Name and Location:")
print(combined_users_by_name.head(), len(combined_users_by_name))

print("\nCombined Users by ID:")
print(combined_users_by_id.head(), len(combined_users_by_id))

Combined Users by Name and Location:
                                           user_name        location  \
0                                           #3cheers  united kingdom   
1  (5)</a></i> - , connecticut, usa - jun 17, 201...   united states   
2  (5)</a></i> - , connecticut, usa - jun 18, 201...   united states   
3  (5)</a></i> - , connecticut, usa - jun 18, 201...   united states   
4                                         (m)rated99       australia   

   nbr_ratings  nbr_reviews user_id        joined  source  
0            1            0  433350  1.479121e+09       1  
1            1            0   65919  1.197803e+09       1  
2            1            0   65919  1.197803e+09       1  
3            1            0   65919  1.197803e+09       1  
4            1            0    2641  1.008760e+09       1   169270

Combined Users by ID:
  user_id  nbr_ratings  nbr_reviews  user_name       joined       location
0       2           52            0       wade  955533600.0  unit

### Conclusions on merging both dataframes
Originally we had 2 datasets with the following lengths:
- BeerAdvocate: 122425 users
- RateBeer: 50592 users



This comes to a total of 173017 users. (Without taking users on both platforms into account)


From these results, we can see the following:
- Joining with user_name returns a df with a length of 169270 thus 'merging' 3747 users.
- Joining with user_id returns a df with a length of 172968 thus 'merging' 49 users.

Thus, we will continue our study with the combined users using the user_name. 

In [9]:
# Segment Users by Location for combined dataset
combined_users_by_location = combined_users_by_name.groupby('location').agg({
    'nbr_reviews': 'sum',
    'nbr_ratings': 'sum',
    'user_id': 'count',
    'joined': 'mean',
    'source': lambda x: x.mode().iloc[0] if not x.mode().empty else None  # Use pandas mode
}).rename(columns={'user_id': 'num_users'}).reset_index()



# Segment Users by Location for individual datasets
ba_users_by_location = ba_users.groupby('location').agg({
    'nbr_reviews': 'sum',
    'nbr_ratings': 'sum',
    'user_id': 'count',
    'joined': 'mean'
}).rename(columns={'user_id': 'num_users'}).reset_index()

rb_users_by_location = rb_users.groupby('location').agg({
    'nbr_reviews': 'sum',
    'nbr_ratings': 'sum',
    'user_id': 'count',
    'joined': 'mean'
}).rename(columns={'user_id': 'num_users'}).reset_index()

# Print the results for combined dataset and the length of the dataset
print("Combined Users by Location:")
print(combined_users_by_location.head(), len(combined_users_by_location))

# Print the results for individual datasets
print("\nBeerAdvocate Users by Location:")  
print(ba_users_by_location.head(), len(ba_users_by_location))

print("\nRateBeer Users by Location:")
print(rb_users_by_location.head(), len(rb_users_by_location))

Combined Users by Location:
         location  nbr_reviews  nbr_ratings  num_users        joined  source
0        abkhazia            0            6          3  1.411496e+09       1
1     afghanistan            3           42         13  1.380948e+09       1
2         albania            0           11          5  1.257209e+09       1
3         algeria            0           39          2  1.415702e+09       1
4  american samoa           18           22          5  1.378999e+09       0 197

BeerAdvocate Users by Location:
         location  nbr_reviews  nbr_ratings  num_users        joined
0     afghanistan            3           12          5  1.411986e+09
1         albania            0            4          1  1.412676e+09
2  american samoa           18           20          4  1.420043e+09
3         andorra            5            8          3  1.260586e+09
4          angola            4            6          1  1.343124e+09 142

RateBeer Users by Location:
         location  nbr_rev

In [10]:
# Analysis Objectives for combined dataset
# Users Analysis: Number of reviews and ratings submitted by users on each site per country
print("Combined - Number of Reviews and Ratings by Location:")
print(combined_users_by_location[['location', 'nbr_reviews', 'nbr_ratings', 'source']].head())

# Users Analysis: Number of users per country
print("Combined - Number of Users by Location:")
print(combined_users_by_location[['location', 'num_users']].head())

# Print the results for individual datasets
print("BeerAdvocate Users by Location:")
print(ba_users_by_location.head())

print("RateBeer Users by Location:")
print(rb_users_by_location.head())

Combined - Number of Reviews and Ratings by Location:
         location  nbr_reviews  nbr_ratings  source
0        abkhazia            0            6       1
1     afghanistan            3           42       1
2         albania            0           11       1
3         algeria            0           39       1
4  american samoa           18           22       0
Combined - Number of Users by Location:
         location  num_users
0        abkhazia          3
1     afghanistan         13
2         albania          5
3         algeria          2
4  american samoa          5
BeerAdvocate Users by Location:
         location  nbr_reviews  nbr_ratings  num_users        joined
0     afghanistan            3           12          5  1.411986e+09
1         albania            0            4          1  1.412676e+09
2  american samoa           18           20          4  1.420043e+09
3         andorra            5            8          3  1.260586e+09
4          angola            4            6

In [11]:
# Analysis Objectives for individual datasets
# Users Analysis: Number of reviews and ratings submitted by users on each site per country
print("BeerAdvocate - Number of Reviews and Ratings by Location:")
print(ba_users_by_location[['location', 'nbr_reviews', 'nbr_ratings']].head())

print("RateBeer - Number of Reviews and Ratings by Location:")
print(rb_users_by_location[['location', 'nbr_reviews', 'nbr_ratings']].head())

# Users Analysis: Number of users per country
print("BeerAdvocate - Number of Users by Location:")
print(ba_users_by_location[['location', 'num_users']].head())

print("RateBeer - Number of Users by Location:")
print(rb_users_by_location[['location', 'num_users']].head())

BeerAdvocate - Number of Reviews and Ratings by Location:
         location  nbr_reviews  nbr_ratings
0     afghanistan            3           12
1         albania            0            4
2  american samoa           18           20
3         andorra            5            8
4          angola            4            6
RateBeer - Number of Reviews and Ratings by Location:
         location  nbr_reviews  nbr_ratings
0        abkhazia            0            6
1     afghanistan            0           30
2         albania            0            7
3         algeria            0           39
4  american samoa            0            2
BeerAdvocate - Number of Users by Location:
         location  num_users
0     afghanistan          5
1         albania          1
2  american samoa          4
3         andorra          3
4          angola          1
RateBeer - Number of Users by Location:
         location  num_users
0        abkhazia          3
1     afghanistan          8
2         alban

In [12]:
combined_users_by_location.head()

,location,nbr_reviews,nbr_ratings,num_users,joined,source
0,abkhazia,0,6,3,1.411496e+09,1
1,afghanistan,3,42,13,1.380948e+09,1
2,albania,0,11,5,1.257209e+09,1
3,algeria,0,39,2,1.415702e+09,1
4,american samoa,18,22,5,1.378999e+09,0


In [13]:
df = combined_users_by_location
df['log_num_users'] = np.log1p(df['num_users'])  # Apply log(1+x) to handle zero counts

# Create an interactive world map with Plotly
fig = px.choropleth(
    df,
    locations="location",
    locationmode="country names",
    color="log_num_users",  # Use the log scale column for coloring
    hover_name="location",
    hover_data={
        "nbr_reviews": True,
        "nbr_ratings": True,
        "num_users": True,  # Display the original number of users for clarity
        "joined": True,
        "source": True
    },
    title="Interactive Map of Users by Location (Log Scale)"
)

# Update layout for better appearance and log-scaled color bar
fig.update_geos(showcoastlines=True, coastlinecolor="Gray")
fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    coloraxis_colorbar={
        'title': "Number of Users (Log Scale)",
        'tickvals': [np.log1p(val) for val in [1, 10, 100, 1000, 10000, 100000]],  # Log scale ticks
        'ticktext': ['1', '10', '100', '1k', '10k', '100k']
    }
)

# Display the figure
fig.show()
fig.write_html("interactive_map.html")

## LOCATION DISTRIBUTION ANALYSIS

## Consideration
Here in the end we will add some consideration